##  Proyecto 1:
### EDA/GEDA

**Cargando librerías:**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
#from pandas import DataFrame

from matplotlib.ticker import FuncFormatter

In [2]:
#define categorical, numerical and date columns
CAT_COLS = ["dia_semana", "codigo_cierre", "año_cierre", "mes_cierre", "mes", "delegacion_inicio", 
            "incidente_c4", "clas_con_f_alarma", "tipo_entrada", "delegacion_cierre", "hora_creacion",
           "hora_cierre"]

DATE_COLS = ["fecha_creacion", "fecha_cierre"]

NUM_COLS = ["latitud", "longitud"]

In [3]:
#esto irá en el script ingestion .py
def ingest_file(file_name):
    """
    Function to retrieve and return the accidents dataset.
    Parameters:
    -----------
    file_name: str
               Path to the file.
    Returns:
    --------
    df: pandas dataframe
    """
    df = pd.read_csv(file_name)
    return df


def drop_cols(df):
    """
    Function to drop unnnecesary columns in the dataset.
    """
    df.drop(columns = ['folio', 'geopoint'], inplace = True)
    return df



def fill_na(df):
    """
    Function to fill null values in a dataframe.
    """
    #aquí podemos ir agregando más cosas cuando descubramos 
    #cómo imputar valores faltantes para latitud y longitud
    df.fillna({
        'delegacion_inicio': 'No Disponible',
        'delegacion_cierre': 'No Disponible'
              }, inplace = True)
    return df


def categoric_transformation(col,df):
    df[col] = df[col].astype("category")
    return df 

def create_categorical(cols, df):
    """
    Function to transform and prepare the categorical features in the dataset.
    """
    #transform to appropriate data type
    for col in cols: 
        df = categoric_transformation(col, df)
     
    return df


def date_transformation(col,df):
    """
    Function to prepare and transform date-type columns. 
    """
    df[col] = pd.to_datetime(df[col])
    return df

def create_date_cols(cols, df):
    for col in cols:
        df = date_transformation(col, df)
    return df 



def generate_label(df):
    """
    Function to create a new column indicating whether there was
    a false alarm or not. 
    Parameters:
    -----------
    df: pandas dataframe
    
    Returns:
    --------
    df: pandas dataframe
    """
    #transformamos la columna para solo quedarnos con la letra del código
    df["codigo_cierre"] = df["codigo_cierre"].apply(lambda x: x[1])
    df['label'] = np.where(
        (df.codigo_cierre == 'F') | (df.codigo_cierre == 'N'), 1, 0)
    return df 


def basic_preprocessing(path):
    """
    Function to summarize all the preprocessing done to the data.
    Parameters:
    -----------
    path: str
          Path to your file
    
    Returns:
    ---------
    df: pandas dataframe
    """
    df = ingest_file(path) 
    df = drop_cols(df) 
    df = generate_label(df) 
    df = fill_na(df) 
    df = create_categorical(CAT_COLS, df) #transform to appropriate data types
    df = create_date_cols(DATE_COLS, df)
    
    return df

**Cargando archivo y haciendo pre-procesamiento:**

In [4]:
data = basic_preprocessing('/Users/Leo/Documents/MCD/1o/IDS/Proyectos/proyecto1/datos/incidentes-viales-c5.csv')
data.head()

,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,mes,label
0,2016-01-23,22:35:04,Sábado,A,2016-01-24,2016,Enero,00:21:08,VENUSTIANO CARRANZA,accidente-choque sin lesionados,19.422113,-99.084643,EMERGENCIA,BOTÓN DE AUXILIO,VENUSTIANO CARRANZA,1,0
1,2016-01-23,22:50:49,Sábado,A,2016-01-24,2016,Enero,04:40:37,CUAJIMALPA,accidente-choque con lesionados,19.358320,-99.297641,URGENCIAS MEDICAS,BOTÓN DE AUXILIO,CUAJIMALPA,1,0
2,2016-01-24,09:40:11,Domingo,N,2016-01-24,2016,Enero,11:13:03,TLALPAN,accidente-choque sin lesionados,19.217070,-99.219070,EMERGENCIA,LLAMADA DEL 066,TLALPAN,1,1
3,2016-01-24,22:40:57,Domingo,N,2016-01-24,2016,Enero,22:55:51,MAGDALENA CONTRERAS,accidente-choque sin lesionados,19.320580,-99.241010,EMERGENCIA,LLAMADA DEL 066,MAGDALENA CONTRERAS,1,1
4,2016-01-24,04:25:15,Domingo,A,2016-01-24,2016,Enero,05:03:59,MIGUEL HIDALGO,accidente-choque sin lesionados,19.452900,-99.215870,EMERGENCIA,LLAMADA DEL 066,MIGUEL HIDALGO,1,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1383138 entries, 0 to 1383137
Data columns (total 17 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   fecha_creacion     1383138 non-null  datetime64[ns]
 1   hora_creacion      1383138 non-null  category      
 2   dia_semana         1383138 non-null  category      
 3   codigo_cierre      1383138 non-null  category      
 4   fecha_cierre       1383138 non-null  datetime64[ns]
 5   año_cierre         1383138 non-null  category      
 6   mes_cierre         1383138 non-null  category      
 7   hora_cierre        1383138 non-null  category      
 8   delegacion_inicio  1383138 non-null  category      
 9   incidente_c4       1383138 non-null  category      
 10  latitud            1382695 non-null  float64       
 11  longitud           1382703 non-null  float64       
 12  clas_con_f_alarma  1383138 non-null  category      
 13  tipo_entrada       1383138 

**Generando Tablas *de data profiling*:**

In [8]:
# variables numéricas
number_variables = data.select_dtypes(include = 'number').columns.values
(len(number_variables), number_variables)

(3, array(['latitud', 'longitud', 'label'], dtype=object))

##  La variable `label` es numérica o categórica?

In [11]:
# Fechas:
dates_variables = data.select_dtypes(include = 'datetime').columns.values
(len(dates_variables), dates_variables)

(2, array(['fecha_creacion', 'fecha_cierre'], dtype=object))

In [12]:
# Variables categóricas:
category_variables = data.select_dtypes(include = 'category').columns.values
(len(category_variables), category_variables)

(12,
 array(['hora_creacion', 'dia_semana', 'codigo_cierre', 'año_cierre',
        'mes_cierre', 'hora_cierre', 'delegacion_inicio', 'incidente_c4',
        'clas_con_f_alarma', 'tipo_entrada', 'delegacion_cierre', 'mes'],
       dtype=object))

In [15]:
# Verificar si tenemos todas las columnas:
if (sum([len(number_variables), len(category_variables), 
      len(dates_variables)]) == data.shape[1]): 
    print('OK')
else:
    print('columns missing')

OK


***Data profiling* de variables categóricas:**

In [20]:
def get_repeated_values(df, col, top):
    top_5 = df.groupby([col])[col]\
                    .count()\
                    .sort_values(ascending = False)\
                    .head(3)
    indexes_top_5 = top_5.index
    
    if ((top == 1) and (len(indexes_top_5) > 0)):
        return indexes_top_5[0]
    elif ((top == 2) and (len(indexes_top_5) > 1)):
        return indexes_top_5[1]
    elif ((top == 3) and (len(indexes_top_5) > 2)):
        return indexes_top_5[2]
    else: 
        return 'undefined'

In [28]:
def category_profiling(df, col):
    """
    Profiling for categoric columns. 
    
    :param: column to analyze
    :return: dictionary
    """
    profiling = {}

    profiling.update({'mode': df[col].mode().values,
                     'num_categories': df[col].nunique(),
                     'categories': df[col].unique(),
                     'uniques': df[col].nunique(),
                     'missings': df[col].isna().sum()/df[col].size*100,
                     'top1_repeated': get_repeated_values(df, col, 1),
                     'top2_repeated': get_repeated_values(df, col, 2),
                     'top3_repeated': get_repeated_values(df, col, 3)})
    
    return profiling

In [31]:
category_profiling_output = {elem: category_profiling(data, elem)\
                             for elem in category_variables}

In [23]:
category_profiling_output
df_category_profiling = pd.DataFrame(category_profiling_output).reset_index()
df_category_profiling.rename(columns={'index': 'metric'}, inplace=True)
df_category_profiling

,metric,hora_creacion,dia_semana,codigo_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,clas_con_f_alarma,tipo_entrada,delegacion_cierre,mes
0,mode,"['20:44:00'] Categories (105887, object): ['0....","['Viernes'] Categories (7, object): ['Domingo'...","['A'] Categories (5, object): ['A', 'D', 'F', ...","[2018] Categories (7, int64): [2014, 2015, 201...","['Octubre'] Categories (12, object): ['Abril',...","['21:59:00'] Categories (106827, object): ['0....","['IZTAPALAPA'] Categories (17, object): ['ALVA...",['accidente-choque sin lesionados'] Categories...,"['EMERGENCIA'] Categories (4, object): ['DELIT...","['LLAMADA DEL 911'] Categories (9, object): ['...","['IZTAPALAPA'] Categories (17, object): ['ALVA...","[10] Categories (12, int64): [1, 2, 3, 4, ...,..."
1,num_categories,105887,7,5,7,12,106827,17,26,4,9,17,12
2,categories,"['22:35:04', '22:50:49', '09:40:11', '22:40:57...","['Sábado', 'Domingo', 'Lunes', 'Martes', 'Miér...","['A', 'N', 'D', 'F', 'I'] Categories (5, objec...","[2016, 2017, 2018, 2019, 2015, 2014, 2020] Cat...","['Enero', 'Febrero', 'Marzo', 'Septiembre', 'O...","['00:21:08', '04:40:37', '11:13:03', '22:55:51...","['VENUSTIANO CARRANZA', 'CUAJIMALPA', 'TLALPAN...","['accidente-choque sin lesionados', 'accidente...","['EMERGENCIA', 'URGENCIAS MEDICAS', 'FALSA ALA...","['BOTÓN DE AUXILIO', 'LLAMADA DEL 066', 'RADIO...","['VENUSTIANO CARRANZA', 'CUAJIMALPA', 'TLALPAN...","[1, 2, 3, 9, 10, ..., 4, 8, 12, 7, 11] Length:..."
3,uniques,105887,7,5,7,12,106827,17,26,4,9,17,12
4,missings,0,0,0,0,0,0,0,0,0,0,0,0
5,top1_repeated,20:44:00,Viernes,A,2018,Octubre,21:59:00,IZTAPALAPA,accidente-choque sin lesionados,EMERGENCIA,LLAMADA DEL 911,IZTAPALAPA,10
6,top2_repeated,19:16:00,Sábado,D,2019,Agosto,21:52:00,GUSTAVO A. MADERO,accidente-choque con lesionados,URGENCIAS MEDICAS,LLAMADA DEL 066,GUSTAVO A. MADERO,8
7,top3_repeated,18:38:00,Jueves,N,2017,Septiembre,21:58:00,CUAUHTEMOC,lesionado-atropellado,FALSA ALARMA,BOTÓN DE AUXILIO,CUAUHTEMOC,9


**Puntos más relevantes:**
1. 
2.  
3.  

***Data profiling* de variables numéricas:**

In [39]:
def numeric_profiling(df, col):
    """
    Profiling for numeric columns. 
    
    :param: column to analyze
    :return: dictionary
    """
    profiling = {}

    profiling.update({'max': df[col].max(),
                     'min': df[col].min(),
                     'mean': df[col].mean(),
                     'stdv': df[col].std(),
                     '25%': df[col].quantile(.25),
                     'median': df[col].median(),
                     '75%': df[col].quantile(.75),
                     'kurtosis': df[col].kurt(),
                     'skewness': df[col].skew(),
                     'uniques': df[col].nunique(),
                     'prop_missings': df[col].isna().sum()/df.shape[0]*100,
                     'top1_repeated': get_repeated_values(df, col, 1),
                     'top2_repeated': get_repeated_values(df, col, 2),
                     'top3_repeated': get_repeated_values(df, col, 3)})
    
    
    return profiling

In [40]:
profiling_dict = {}

numeric_profiling_output = {elem: numeric_profiling(data, elem)\
                            for elem in number_variables}

In [41]:
df_numeric_profiling = pd.DataFrame(numeric_profiling_output).reset_index()
df_numeric_profiling.rename(columns={'index': 'metric'}, inplace=True)
df_numeric_profiling

,metric,latitud,longitud,label
0,max,195.303000,-98.945370,1
1,min,19.094020,-991.764000,0
2,mean,19.383909,-99.143590,0.204103
3,stdv,0.266638,2.399677,0.403045
4,25%,19.336850,-99.179300,0
5,median,19.384080,-99.140230,0
6,75%,19.434960,-99.096000,0
7,kurtosis,400835.233813,138084.253218,0.15593
8,skewness,611.376322,-371.476884,1.46831
9,uniques,82501.000000,78984.000000,2


**Puntos más relevantes:**
1. 
2.  
3.  

##  La variable `label` es numérica o categórica?

***Data profiling* de variables de fecha:**

In [36]:
def dates_profiling(df, col):
    """
    Profiling for categoric columns. 
    
    :param: column to analyze
    :return: dictionary
    """
    profiling = {}

    profiling.update({'mode': df[col].mode().values,
                     'num_dates': df[col].nunique(),
                     'dates': df[col].unique(),
                     'uniques': df[col].nunique(),
                     'missings': df[col].isna().sum()/df[col].size*100,
                     'top1_repeated': get_repeated_values(df, col, 1),
                     'top2_repeated': get_repeated_values(df, col, 2),
                     'top3_repeated': get_repeated_values(df, col, 3)})
    
    return profiling

In [37]:
dates_profiling_output = {elem: dates_profiling(data, elem)\
                             for elem in dates_variables}

In [38]:
dates_profiling_output
df_dates_profiling = pd.DataFrame(dates_profiling_output).reset_index()
df_dates_profiling.rename(columns={'index': 'metric'}, inplace=True)
df_dates_profiling

,metric,fecha_creacion,fecha_cierre
0,mode,[2020-02-14T00:00:00.000000000],[2020-02-14T00:00:00.000000000]
1,num_dates,2497,2496
2,dates,"[2016-01-23T00:00:00.000000000, 2016-01-24T00:...","[2016-01-24T00:00:00.000000000, 2016-01-25T00:..."
3,uniques,2497,2496
4,missings,0,0
5,top1_repeated,2020-02-14 00:00:00,2020-02-14 00:00:00
6,top2_repeated,2018-10-26 00:00:00,2017-08-12 00:00:00
7,top3_repeated,2019-11-30 00:00:00,2017-02-12 00:00:00


**Puntos más relevantes:**
1. 
2.  
3.  

**Cargando datos originales:**

In [2]:
df = pd.read_csv("datos/incidentes-viales-c5.csv")

**Obteniendo infromación general del dataset**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1383138 entries, 0 to 1383137
Data columns (total 18 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   folio              1383138 non-null  object 
 1   fecha_creacion     1383138 non-null  object 
 2   hora_creacion      1383138 non-null  object 
 3   dia_semana         1383138 non-null  object 
 4   codigo_cierre      1383138 non-null  object 
 5   fecha_cierre       1383138 non-null  object 
 6   año_cierre         1383138 non-null  int64  
 7   mes_cierre         1383138 non-null  object 
 8   hora_cierre        1383138 non-null  object 
 9   delegacion_inicio  1382980 non-null  object 
 10  incidente_c4       1383138 non-null  object 
 11  latitud            1382695 non-null  float64
 12  longitud           1382703 non-null  float64
 13  clas_con_f_alarma  1383138 non-null  object 
 14  tipo_entrada       1383138 non-null  object 
 15  delegacion_cierre  1382998 non-n

**Muestra de datos:**

In [4]:
df.head(5)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes
0,GA/160123/05714,23/01/2016,22:35:04,Sábado,(A) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,00:21:08,VENUSTIANO CARRANZA,accidente-choque sin lesionados,19.422113,-99.084643,EMERGENCIA,BOTÓN DE AUXILIO,VENUSTIANO CARRANZA,"19.422113,-99.084643",1
1,AO/160123/05826,23/01/2016,22:50:49,Sábado,(A) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,04:40:37,CUAJIMALPA,accidente-choque con lesionados,19.358320,-99.297641,URGENCIAS MEDICAS,BOTÓN DE AUXILIO,CUAJIMALPA,"19.35832,-99.297641",1
2,C4/160124/02187,24/01/2016,09:40:11,Domingo,(N) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,11:13:03,TLALPAN,accidente-choque sin lesionados,19.217070,-99.219070,EMERGENCIA,LLAMADA DEL 066,TLALPAN,"19.21707,-99.21907008",1
3,C4/160124/05723,24/01/2016,22:40:57,Domingo,(N) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,22:55:51,MAGDALENA CONTRERAS,accidente-choque sin lesionados,19.320580,-99.241010,EMERGENCIA,LLAMADA DEL 066,MAGDALENA CONTRERAS,"19.32057999,-99.24100992",1
4,C4/160124/01334,24/01/2016,04:25:15,Domingo,(A) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,05:03:59,MIGUEL HIDALGO,accidente-choque sin lesionados,19.452900,-99.215870,EMERGENCIA,LLAMADA DEL 066,MIGUEL HIDALGO,"19.45289997,-99.21587004",1


**Resumen:**

1. ¿Cuántas variables tenemos? ***18***

2. ¿Cuántas observaciones tenemos? ***1,383,138***

3. ¿Cuántas variables numéricas tenemos? ***4 variables numéricas***  (año_cierre, latitud, longitud, mes)
    -  ¿Cambiar año y mes a categórica?

4. ¿Cuántas variables de fecha tenemos? ***2 de fecha***  (fecha_creacion, fecha_cierre)

5. ¿Cuántas variables categóricas tenemos? ***8 variables categóricas***  (dia_semana, codigo_cierre, mes_cierre, delegacion_inicio, incidente_c4, clas_con_f_alarma, tipo_entrada, delegacion_cierre)

6. ¿Cuántas variables de texto tenemos? ***4 variable de texto***  (folio, hora_creacion, hora_cierre, geopoint)
    -  ¿Eliminar geopoint?

7. ¿Cuántas observaciones únicas tenemos por variable?: 

In [5]:
def unicos_val_by_col(df):
    #Obtiene los valores únicos por columa, recibe un dataframe.
    for col in list(df):
        print(col + ": " , len(df[col].unique().tolist()))
        
unicos_val_by_col(df)

folio:  1383138
fecha_creacion:  2637
hora_creacion:  105887
dia_semana:  7
codigo_cierre:  5
fecha_cierre:  2621
año_cierre:  7
mes_cierre:  12
hora_cierre:  106827
delegacion_inicio:  17
incidente_c4:  26
latitud:  82502
longitud:  78985
clas_con_f_alarma:  4
tipo_entrada:  9
delegacion_cierre:  17
geopoint:  134289
mes:  12


**Observaciones:**

-  Hay 1,383,138 folios.
-  Hay datos de 7 años (2014 - 2020).
-  Hay 17 delegaciones en vez de 16, tanto de inicio como de cierre.
    -  Sí son 16, ¿por qué la función indica 17?
-  Sí coinciden los códigos de cierre (5) y las clasificaciones (4) con el diccionario.
-  Hay 26 tipos de incidente, contra 25 que muestra el diccionario.
-  Hay 9 tipos de entrada, contra 7 que muestra el diccionario.

###  Profiling inicial:

In [6]:
def describe_fast(df, p):
    #Esta funcion arroja el resumen de información por tipo de variable
    # El valor p es 1 para variables categoricas, 0 para variables numericas
    
    cat = df.select_dtypes(np.object)
    num = df.select_dtypes(np.number)
    n = len(cat.columns)
    m = len(num.columns)
    
    if p== 1: 
        
        for i in range(0,n):
            print("Variable:", cat.columns[i])
            print("Número de registros únicos/categorías:",len(cat.iloc[:,i].unique())) 
            print("Porcentaje de valores nulos:", sum(pd.isna(cat.iloc[:,1]))/len(cat.iloc[:,1]))
            print("Distribución:")
            print(cat.iloc[:,i].value_counts()/ len(cat.iloc[:,i]),"\n")
    else:
        for i in range(0,m):
            print(num.iloc[:,i].describe(),)
            print("Número de registros únicos:", len(num.iloc[:,i].unique()))
            print("Porcentaje de valores nulos:", sum(pd.isna(num.iloc[:,i]))/len(num.iloc[:,i]),"\n")

**Variables numéricas:**

In [7]:
describe_fast(df, 2)

count    1.383138e+06
mean     2.017023e+03
std      1.869989e+00
min      2.014000e+03
25%      2.015000e+03
50%      2.017000e+03
75%      2.019000e+03
max      2.020000e+03
Name: año_cierre, dtype: float64
Número de registros únicos: 7
Porcentaje de valores nulos: 0.0 

count    1.382695e+06
mean     1.938391e+01
std      2.666382e-01
min      1.909402e+01
25%      1.933685e+01
50%      1.938408e+01
75%      1.943496e+01
max      1.953030e+02
Name: latitud, dtype: float64
Número de registros únicos: 82502
Porcentaje de valores nulos: 0.0003202861898089706 

count    1.382703e+06
mean    -9.914359e+01
std      2.399677e+00
min     -9.917640e+02
25%     -9.917930e+01
50%     -9.914023e+01
75%     -9.909600e+01
max     -9.894537e+01
Name: longitud, dtype: float64
Número de registros únicos: 78985
Porcentaje de valores nulos: 0.00031450224055734133 

count    1.383138e+06
mean     6.617056e+00
std      3.460213e+00
min      1.000000e+00
25%      4.000000e+00
50%      7.000000e+00
75%   

**Observaciones:**

-  Los años van del 2014 al 2020 (7 en total).
-  ¿Cuáles son los años con más folios totales?
-  ¿Cuáles son los años con más folios falsos?
-  ¿Cuáles son los meses con más folios totales?
-  ¿Cuáles son los meses con más folios falsos?
-  Mapa con folios y distribución de llamadas falsas y verdaderas (latitud y longitud).

**Variables categóricas:**

In [9]:
describe_fast(df, 1)

Variable: folio
Número de registros únicos/categorías: 1383138
Porcentaje de valores nulos: 0.0
Distribución:
C4/141007/01165    7.229937e-07
C4/150611/00945    7.229937e-07
C5/161122/03371    7.229937e-07
C5/191006/08209    7.229937e-07
C5/180728/00574    7.229937e-07
                       ...     
C5/190521/08519    7.229937e-07
C5/170703/01579    7.229937e-07
C5/180721/06550    7.229937e-07
C5/181124/00769    7.229937e-07
C5/190415/07143    7.229937e-07
Name: folio, Length: 1383138, dtype: float64 

Variable: fecha_creacion
Número de registros únicos/categorías: 2637
Porcentaje de valores nulos: 0.0
Distribución:
14/02/2020    8.408416e-04
26/10/2018    7.432375e-04
08/12/2017    7.309466e-04
30/11/2019    7.309466e-04
21/04/2018    7.295006e-04
                  ...     
04/03/2019    7.229937e-07
13/07/2019    7.229937e-07
06/08/2019    7.229937e-07
21/03/2019    7.229937e-07
27/08/2019    7.229937e-07
Name: fecha_creacion, Length: 2637, dtype: float64 

Variable: hora_creacion
N

Número de registros únicos/categorías: 134289
Porcentaje de valores nulos: 0.0
Distribución:
19.37168001,-99.08024004    1.563112e-03
19.30431996,-99.08714016    1.513949e-03
19.347021,-99.180646        1.250779e-03
19.35663003,-99.08655984    1.082322e-03
19.36316997,-99.05739984    1.017252e-03
                                ...     
19.56454,-99.13759          7.229937e-07
19.19197845,-99.0226593     7.229937e-07
19.28632791,-99.23747174    7.229937e-07
19.46759,-99.09793          7.229937e-07
19.34632833,-99.17717       7.229937e-07
Name: geopoint, Length: 134288, dtype: float64 



**Observaciones:**

-  Las llamadas falsas (N + F) equivalen al 20.3% de las llamadas totales.
-  El día con más llamadas es el 14/feb/2020.  
    -  ¿Hay alguna fecha con más proporción de llamadas falsas?
    -  ¿Cuál es la distribución de llamadas falsas y verdaderas por año, mes, día de la semana, y hora?
-  La delegación con más llamadas es Iztapalapa (16%), Milpa Alta es la que menos tiene (0.05%).    
    -  ¿Cuál es la distribución de llamadas falsas y verdaderas para cada delegación?
-  ¿Nos interesa la relación (tiempo y/o delegación), entre la creación y el cierre?
-  El tipo de incidente (incidente_c4), ¿es un dato de entrada o de salida?  
    -  ¿Cuál es la proporción de llamadas falsas?
-  La clasificación (clas_con_f_alarma), ¿es un dato de entrada o de salida?
    -  ¿Cuál es la proporción de llamadas falsas?
-  El tipo de entrada más común es "Llamada del 911" (53.4%), el menos común es "Aplicativos" (0.006%)
    -  ¿Cuál es la distribución de llamadas falsas y verdaderas para cada tipo de entrada?
-  Hay 16 delegaciones, ¿el profiling dice que son 17?
-  ¿Hay ubicaciones específicas con un alto índice de llamadas falsas?

### Gráficas:

-  Mapa con llamadas falsas y verdaderas.  (O)
    -  ¿Zoom en el mapa con alguna zona de alta incidencia de llamadas falsas?
-  Gráfica de barras con proporción de la etiqueta (llamadas falsas y verdaderas).  (O)
-  Gráfica con proporción de llamadas falsas y verdaderas por año.
-  Gráfica con proporción de llamadas falsas y verdaderas por mes.
-  Gráfica con proporción de llamadas falsas y verdaderas por día de la semana.
-  Gráfica con proporción de llamadas falsas y verdaderas por cada 3 horas.
-  Gráfica con proporción de llamadas falsas y verdaderas por cada tipo de entrada.
-  Gráfica con proporción de llamadas falsas y verdaderas por delegación.